In [1]:
import os, codecs, string
import xml.etree.ElementTree as ET
import ntpath
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Load Translation Memory file 
***We will use the 'DGT-TM: A freely Available Translation Memory in 22 Languages'***

(https://ec.europa.eu/jrc/en/language-technologies/dgt-translation-memory)
(http://www.lrec-conf.org/proceedings/lrec2012/pdf/814_Paper.pdf)

In [2]:
# Load and Parse the TM:
TMpath = r'data\Vol_2016_9\test.tmx'
tree = ET.parse(TMpath)
root = tree.getroot()
print(root.items())

[('version', '1.4')]


In [3]:
root.tag

'tmx'

In [4]:
# LET'S GET A TOTAL COUNT OF THE SEGMENTS:
tm_dict = set()
for x in root.iter('tu'):
    for i in x.getchildren():
        for txt in i.getchildren():
            txt = str(txt.text).replace("\n", " ")
            tm_dict.add(str(txt))
len(tm_dict)

1156

In [5]:
#LET'S SAMPLE THE 1st SEGMENT:
print(list(tm_dict)[0])

Ņemot vērā visu iepriekš minēto, Aģentūra ieteica, ka Regulas (EK) Nr. 216/2008 11. panta 1. punktu beidz piemērot HATA izsniegtajām atzīšanas apliecībām par eksāmenu pamatzināšanās par tehniskajiem moduļiem un Part-66 apliecībām, ko izdevušas dalībvalstu kompetentās iestādes, pamatojoties uz šīm atzīšanas apliecībām.


In [6]:
# LET'S GET TOTAL COUNT AND COUNT OF 1 SIDE:
tm_dic = {}
cnt = 0
test_set = set()
for x in root.iter('tu'):
    k =None
    v = None
    for i in x.getchildren():
        for txt in i.getchildren():
            txt = str(txt.text).replace("\n", " ").strip()
            if cnt%2==0:
                k = txt
                test_set.add(txt)
            else:
                v = txt
                tm_dic[k] = v
            cnt+=1
print(cnt)
print(len(test_set))

1172
578


In [7]:
# CHECK FOR PLACEHOLDERS (</>):
cnt = 0
for k,v in tm_dic.items():
    #if v.lower().count('[')>0:
    if v.count('<')>0 and v.count('>')>0:
        cnt+=1
        print(k,"\n")
print(cnt)

len(tm_dic)

0


551

In [8]:
# REMOVE NUMERIC SEGMENTS:
for k in tm_dic.copy():
    if k:
        if k.isnumeric():
            #del tm_dic[k]
            print(k)
    else:
        del tm_dic[k]

In [9]:
# DEFINE FUNCTIONS TO REMOVE TEXT IN SQUARE BRACKETS (Translator Comments) 
print("Length of data before filtering", str(len(tm_dic)))
def rmvTxt_inSqBrckts(text):
    #text = re.sub("[\(\[].*?[\)\]]", " ", text, 1) # Removes text in parens and sq brackets, 1st occurrence.
    text = re.sub("[\[].*?[\]]", " ", text) # Removes text in sq brackets
    text = re.sub(" +", " ", text) # Removes excess spaces
    return text.strip()

def rmvTxt_placeholders(text):
    #text = re.sub("[\(\[].*?[\)\]]", " ", text, 1) # Removes text in parens and sq brackets, 1st occurrence.
    text = re.sub("[\<].*?[\>]", " ", text) # Removes text in between arrow brackets
    text = re.sub(" +", " ", text) # Removes excess spaces
    return text.strip()

cnt = 0
cnt2 = 0
for k,v in tm_dic.copy().items():
    if k.count('<')>0 and k.count('>')>0:
        tm_dic[rmvTxt_placeholders(k)] = rmvTxt_placeholders(v)
        del dic[k]
        cnt+=1
    if v.lower().count("[tc:")>0:
        tm_dic[k] = rmvTxt_inSqBrckts(v)
        cnt2+=1
        
print("Length of data after filtering", str(len(tm_dic)))
print("Number of Placeholders filtered from data", str(cnt))
print("Number of TC Comments filtered from data", str(cnt2))

Length of data before filtering 550
Length of data after filtering 550
Number of Placeholders filtered from data 0
Number of TC Comments filtered from data 0


In [10]:
# Define a function to split the texts into tokens and remove special characters and stopwords:
special_chars = [  # Source = https://towardsdatascience.com/multilingual-document-classification-2731ce8c0163
    ',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '/', 
    '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '·', 
    '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '™', '›','♥',
    '←', '×', '§', '″', '′', '█', '½', '…', '“', '★', '”','–', '●', 
    '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾','═', '¦',
    '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '⊕', '▼', '▪',  
    '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '∞',
    '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³',
    '・', '╦', '╣', '╔', '╗', '▬','❤', 'ï', 'Ø', '¹', '≤', '‡', '√'
]

def char_checktext(text):
    
    if set(text.strip().lower().replace(' ','')).issubset(''.join(special_chars) + string.whitespace + string.digits):
        return True
    else:
        return False

print("Length of data before trimming", str(len(tm_dic)))
cnt = 0 
for k in tm_dic.copy():
    if char_checktext(k):
        del tm_dic[k]
        cnt+=1
print("Length of data after trimming", str(len(tm_dic)))
print("Number of segments filtered from data", str(cnt))

Length of data before trimming 550
Length of data after trimming 550
Number of segments filtered from data 0


In [11]:
for k in tm_dic:
    if k.isnumeric():
        #del tm_dic[k]
        print(k)
len([x for x in tm_dic.keys() if x.strip() != ''])

550

In [12]:
# PREPARE TO CONVERT DICTIONARY TO PANDAS DATAFRAME BY CONVERTING VALUES TO LIST TYPE:
for k,v in tm_dic.items():
    tm_dic[k]=[v]

In [13]:
# SAMPLE THE DATA:
for k,v in tm_dic.items():
    print(k, "\n", v)
    break

EFTA Surveillance Authority Decision 
 ['Beschluss der EFTA-Überwachungsbehörde']


In [14]:
# CONVERT DICTIONARY TO DATAFRAME:
data = pd.DataFrame(tm_dic)

In [15]:
# PROPERLY FORMAT DATAFRAME:
data = data.T
data.reset_index(inplace=True)
data.rename(columns={'index':'en', 0:'de'}, inplace = True)
data.head(1)

,en,de
0,EFTA Surveillance Authority Decision,Beschluss der EFTA-Überwachungsbehörde


In [16]:
len(data)

550

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   en      550 non-null    object
 1   de      550 non-null    object
dtypes: object(2)
memory usage: 8.7+ KB


In [18]:
for i in data.en.isnull():
    if i==True:
        print(i)

In [19]:
# EXPORT DATA TO .csv:
path_csv = r'data\parallel_corpus.csv'
data.to_csv(path_csv)

In [20]:
# TEST RELOADING DATAFRAME FROM .csv:
data_prepped = pd.read_csv(path_csv)
data_prepped = data.drop(columns=data.columns[0])
data.head(1)

,en,de
0,EFTA Surveillance Authority Decision,Beschluss der EFTA-Überwachungsbehörde
